# Plot gridded SWE metrics by elevation band and canopy cover percent

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle
import altair as alt
import xarray as xr
from shapely import Polygon
import matplotlib.pyplot as plt
#import contextily as ctx
import rioxarray as rxr

### Load Data:

In [2]:
swe = xr.open_dataset('../../data/reanalysis/swe_reanalysis_ucrb.nc')

ucrb = pickle.load(open('../../data/misc/ucrb.pkl', 'rb'))

In [3]:
# Mean of all stations in the UCRB
calculated_swe_mean_snv_ucrb_all = xr.open_dataset('../../data/calculated_swe/snv/calculated_swe_mean_snv_ucrb_all.nc')
calculated_swe_quantiles_median_all = xr.open_dataset('../../data/calculated_swe/quantiles/calculated_swe_quantile_median_all_q8.nc')
calculated_swe_quantiles_mean_all = xr.open_dataset('../../data/calculated_swe/quantiles/calculated_swe_quantile_mean_all_q8.nc')
# Mean of within-station clusters, by different cluster definitions
calculated_swe_mean_snv_wus_coarse = xr.open_dataset('../../data/calculated_swe/snv/calculated_swe_mean_snv_wus_coarse2.nc')
calculated_swe_quantiles_wus_coarse_q8 = xr.open_dataset('../../data/calculated_swe/quantiles/calculated_swe_quantile_median_wic_wus_coarse_q8_2.nc')
calculated_swe_quantiles_wus_coarse_q8_mean = xr.open_dataset('../../data/calculated_swe/quantiles/calculated_swe_quantile_mean_wic_wus_coarse_q8_2.nc')

# Nearest station
calculated_swe_nearest_snv_nc = xr.open_dataset('../../data/calculated_swe/snv/calculated_swe_nearest_snv_nc.nc')
calculated_swe_nearest_station_quantile_q8 = xr.open_dataset('../../data/calculated_swe/quantiles/calculated_swe_nearest_station_quantile_q8.nc')

# Model
snowmodel = xr.open_dataset('../../data/snowmodel/sm_swed_ucrb_match_2000_2021.nc', decode_coords="all")

In [4]:
snowmodel_dem = xr.open_dataset('../../data/snowmodel/sm_swed_04_01_2000_2021.nc', decode_coords="all")
snowmodel_dem = snowmodel_dem.transpose('Year', 'y', 'x')
snowmodel_dem = snowmodel_dem.hgt
snowmodel_dem = snowmodel_dem.rio.write_crs('epsg:4326')
snowmodel_dem = snowmodel_dem.rio.clip(ucrb.geometry)
snowmodel_dem = snowmodel_dem.rio.reproject_match(swe.SWE_Post[0])

In [5]:
# canopy = xr.open_dataset('../../data/canopy/rcmap_tree_2009.tif')
# canopy_coarsened = canopy.coarsen(x=10, y=10,boundary="trim").mean()
# canopy_coarsened_clip = canopy_coarsened.rio.clip(ucrb_section_trans.geometry)
# canopy_coarsened_clip_trans = canopy_coarsened_clip.rio.reproject('epsg:4326')
# canopy_coarsened_clip_trans_ucrb = canopy_coarsened_clip_trans.rio.clip(ucrb.geometry)
# canopy_ucrb = canopy_coarsened_clip_trans_ucrb.rio.reproject_match(swe.SWE_Post[0])
canopy_ucrb = xr.open_dataset('../../data/canopy/canopy_cover_ucrb.nc')

In [6]:
stations = pickle.load(open('../../data/snv_dataframes/snv_temp_precip_cluster_gdf_snvs_quantiles.pkl', 'rb'))

/var/folders/s4/pymynk_14w75x2ghv0xjkqlm0000gn/T/ipykernel_6073/621404023.py:1: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  stations = pickle.load(open('../../data/snv_dataframes/snv_temp_precip_cluster_gdf_snvs_quantiles.pkl', 'rb'))


In [7]:
stations_1yr = stations[stations.index.get_level_values(1)==1990]
stations_1yr = stations_1yr.clip(ucrb)

In [8]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def calculate_metrics_elev(calculated_swe, band, version = 'Median quantile of w/in-cluster stations', method= 'Quantile'):
    dif_swe = calculated_swe.where(swe.SWE_Post.mean(dim='Year')>0.1) - swe.SWE_Post.where(swe.SWE_Post.mean(dim='Year')>0.1)
    R = []
    MD = []
    RMSE = []
    for year in np.arange(0,32):
            
        sr_array = swe.SWE_Post[year].where((snowmodel_dem >= band[0]) & (snowmodel_dem < band[1]) & (swe.SWE_Post.mean(dim='Year')>0.1) & calculated_swe[year].notnull()).values.flatten()
        sr_array = sr_array[~np.isnan(sr_array)]
        calc_array = calculated_swe[year].where((snowmodel_dem >= band[0]) & (snowmodel_dem < band[1]) & (swe.SWE_Post.mean(dim='Year')>0.1) & swe.SWE_Post[year].notnull()).values.flatten()
        calc_array = calc_array[~np.isnan(calc_array)]
        
        R.append(np.corrcoef(sr_array, calc_array)[0,1])
        
        MD.append(np.nanmean(dif_swe[year].where((snowmodel_dem >= band[0]) & (snowmodel_dem < band[1]) & (swe.SWE_Post.mean(dim='Year')>0.1)).values))

        RMSE.append(sqrt(mean_squared_error(sr_array, calc_array)))

    data = {
            "R": R,
            "MD": MD,
            "RMSE": RMSE,
            #'StD': std
            'elevation_band': f'{band[0]}-{band[1]}',
            'year': np.arange(1990,2022),
            #'sca_percent': sca_perc,
            "version":version,
            "method": method
        }

        #load data into a DataFrame object:
    df = pd.DataFrame(data)
        
    return df

In [9]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def calculate_metrics_canopy(calculated_swe, band, version = 'Median quantile of w/in-cluster stations', method= 'Quantile'):
    dif_swe = calculated_swe.where(swe.SWE_Post.mean(dim='Year')>0.1) - swe.SWE_Post.where(swe.SWE_Post.mean(dim='Year')>0.1)
    R = []
    MD = []
    RMSE = []
    for year in np.arange(0,32):
            
        sr_array = swe.SWE_Post[year].where((canopy_ucrb.band_data >= band[0]) & (canopy_ucrb.band_data < band[1]) & (swe.SWE_Post.mean(dim='Year')>0.1) & calculated_swe[year].notnull()).values.flatten()
        sr_array = sr_array[~np.isnan(sr_array)]
        calc_array = calculated_swe[year].where((canopy_ucrb.band_data >= band[0]) & (canopy_ucrb.band_data < band[1]) & (swe.SWE_Post.mean(dim='Year')>0.1) & swe.SWE_Post[year].notnull()).values.flatten()
        calc_array = calc_array[~np.isnan(calc_array)]
        
        R.append(np.corrcoef(sr_array, calc_array)[0,1])
        
        MD.append(np.nanmean(dif_swe[year].where((canopy_ucrb.band_data >= band[0]) & (canopy_ucrb.band_data < band[1]) & (swe.SWE_Post.mean(dim='Year')>0.1)).values))

        RMSE.append(sqrt(mean_squared_error(sr_array, calc_array)))

    data = {
            "R": R,
            "MD": MD,
            "RMSE": RMSE,
            #'StD': std
            'canopy_band': f'{band[0]}-{band[1]}',
            'year': np.arange(1990,2022),
            #'sca_percent': sca_perc,
            "version":version,
            "method": method
        }

        #load data into a DataFrame object:
    df = pd.DataFrame(data)
        
    return df

In [10]:
elevation_bands = [[1500,2500], [2500,3000], [3000,3500], [3500,4000], [4000,4500]]

#quantile_median_elev_2000 = calculate_metrics_elev(calculated_swe_quantiles_wus_coarse_q8.SWE_Post, band=elevation_bands[0], version = 'Median quantile of w/in-cluster stations', method= 'Quantile')

quantile_median_elev_2500 = calculate_metrics_elev(calculated_swe_quantiles_wus_coarse_q8.SWE_Post, band=elevation_bands[0], version = 'Median quantile of w/in-cluster stations', method= 'Quantile')

quantile_median_elev_3000 = calculate_metrics_elev(calculated_swe_quantiles_wus_coarse_q8.SWE_Post, band=elevation_bands[1], version = 'Median quantile of w/in-cluster stations', method= 'Quantile')

quantile_median_elev_3500 = calculate_metrics_elev(calculated_swe_quantiles_wus_coarse_q8.SWE_Post, band=elevation_bands[2], version = 'Median quantile of w/in-cluster stations', method= 'Quantile')

quantile_median_elev_4000 = calculate_metrics_elev(calculated_swe_quantiles_wus_coarse_q8.SWE_Post, band=elevation_bands[3], version = 'Median quantile of w/in-cluster stations', method= 'Quantile')

quantile_median_elev_4500 = calculate_metrics_elev(calculated_swe_quantiles_wus_coarse_q8.SWE_Post, band=elevation_bands[4], version = 'Median quantile of w/in-cluster stations', method= 'Quantile')

In [11]:
elevation_bands = [[1500,2500], [2500,3000], [3000,3500], [3500,4000], [4000,4500]]

#snv_mean_elev_2000 = calculate_metrics_elev(calculated_swe_mean_snv_wus_coarse.SWE_Post, band=elevation_bands[0], version = 'Mean SNV of w/in-cluster stations', method= 'SNV')

snv_mean_elev_2500 = calculate_metrics_elev(calculated_swe_mean_snv_wus_coarse.SWE_Post, band=elevation_bands[0], version = 'Mean SNV of w/in-cluster stations', method= 'SA')

snv_mean_elev_3000 = calculate_metrics_elev(calculated_swe_mean_snv_wus_coarse.SWE_Post, band=elevation_bands[1], version = 'Mean SNV of w/in-cluster stations', method= 'SA')

snv_mean_elev_3500 = calculate_metrics_elev(calculated_swe_mean_snv_wus_coarse.SWE_Post, band=elevation_bands[2], version = 'Mean SNV of w/in-cluster stations', method= 'SA')

snv_mean_elev_4000 = calculate_metrics_elev(calculated_swe_mean_snv_wus_coarse.SWE_Post, band=elevation_bands[3], version = 'Mean SNV of w/in-cluster stations', method= 'SA')

snv_mean_elev_4500 = calculate_metrics_elev(calculated_swe_mean_snv_wus_coarse.SWE_Post, band=elevation_bands[4], version = 'Mean SNV of w/in-cluster stations', method= 'SA')

In [12]:
metrics_elev = pd.concat(
    [
    #quantile_median_elev_2000,
    quantile_median_elev_2500, 
    quantile_median_elev_3000, 
    quantile_median_elev_3500, 
    quantile_median_elev_4000, 
    quantile_median_elev_4500,
    #snv_mean_elev_2000,
    snv_mean_elev_2500,
    snv_mean_elev_3000,
    snv_mean_elev_3500,
    snv_mean_elev_4000,
    snv_mean_elev_4500], 
    ignore_index=True)

In [13]:
elevation_bands = [[1500,2500], [2500,3000], [3000,3500], [3500,4000], [4000,4500]]

area_per_band = []
stations_per_band = []

for band in elevation_bands:
    sr_array = swe.SWE_Post[0].where((snowmodel_dem >= band[0]) & (snowmodel_dem < band[1]) & (swe.SWE_Post.mean(dim='Year')>0.1)).values.flatten()
    sr_array = sr_array[~np.isnan(sr_array)]
    area_per_band.append(len(sr_array)*(500**2)/1000000)
    stations_per_band.append(len(stations_1yr[(stations_1yr['elevation_m']>=band[0]) & (stations_1yr['elevation_m']<band[1])]))

data = {
        "elevation_band": elevation_bands,
        "area_per_band": area_per_band,
        "stations_per_band": stations_per_band,
    }

#load data into a DataFrame object:
area_by_elev = pd.DataFrame(data)

In [14]:
area_by_elev['percent_area'] = (area_by_elev['area_per_band']/77945.25) * 100

In [15]:
area_by_elev['percent_stations'] = (area_by_elev['stations_per_band']/area_by_elev['stations_per_band'].sum())*100

In [16]:
canopy_bands = [[0,20], [20,40], [40,60], [60,80], [80,100]]

#quantile_median_elev_2000 = calculate_metrics_elev(calculated_swe_quantiles_wus_coarse_q8.SWE_Post, band=elevation_bands[0], version = 'Median quantile of w/in-cluster stations', method= 'Quantile')

quantile_median_canopy_20 = calculate_metrics_canopy(calculated_swe_quantiles_wus_coarse_q8.SWE_Post, band=canopy_bands[0], version = 'Median quantile of w/in-cluster stations', method= 'Quantile')

quantile_median_canopy_40 = calculate_metrics_canopy(calculated_swe_quantiles_wus_coarse_q8.SWE_Post, band=canopy_bands[1], version = 'Median quantile of w/in-cluster stations', method= 'Quantile')

quantile_median_canopy_60 = calculate_metrics_canopy(calculated_swe_quantiles_wus_coarse_q8.SWE_Post, band=canopy_bands[2], version = 'Median quantile of w/in-cluster stations', method= 'Quantile')

quantile_median_canopy_80 = calculate_metrics_canopy(calculated_swe_quantiles_wus_coarse_q8.SWE_Post, band=canopy_bands[3], version = 'Median quantile of w/in-cluster stations', method= 'Quantile')

quantile_median_canopy_100 = calculate_metrics_canopy(calculated_swe_quantiles_wus_coarse_q8.SWE_Post, band=canopy_bands[4], version = 'Median quantile of w/in-cluster stations', method= 'Quantile')

In [17]:
canopy_bands = [[0,20], [20,40], [40,60], [60,80], [80,100]]

#quantile_median_elev_2000 = calculate_metrics_elev(calculated_swe_quantiles_wus_coarse_q8.SWE_Post, band=elevation_bands[0], version = 'Median quantile of w/in-cluster stations', method= 'Quantile')

snv_median_canopy_20 = calculate_metrics_canopy(calculated_swe_mean_snv_wus_coarse.SWE_Post, band=canopy_bands[0], version = 'Mean SNV of w/in-cluster stations', method= 'SA')

snv_median_canopy_40 = calculate_metrics_canopy(calculated_swe_mean_snv_wus_coarse.SWE_Post, band=canopy_bands[1], version = 'Mean SNV of w/in-cluster stations', method= 'SA')

snv_median_canopy_60 = calculate_metrics_canopy(calculated_swe_mean_snv_wus_coarse.SWE_Post, band=canopy_bands[2], version = 'Mean SNV of w/in-cluster stations', method= 'SA')

snv_median_canopy_80 = calculate_metrics_canopy(calculated_swe_mean_snv_wus_coarse.SWE_Post, band=canopy_bands[3], version = 'Mean SNV of w/in-cluster stations', method= 'SA')

snv_median_canopy_100 = calculate_metrics_canopy(calculated_swe_mean_snv_wus_coarse.SWE_Post, band=canopy_bands[4], version = 'Mean of w/in-cluster stations', method= 'SA')

In [18]:
metrics_canopy = pd.concat(
    [
    quantile_median_canopy_20, 
    quantile_median_canopy_40, 
    quantile_median_canopy_60, 
    quantile_median_canopy_80, 
    quantile_median_canopy_100,
    snv_median_canopy_20,
    snv_median_canopy_40,
    snv_median_canopy_60,
    snv_median_canopy_80,
    snv_median_canopy_100], 
    ignore_index=True)

In [19]:
xs = stations_1yr.geometry.x.values
ys = stations_1yr.geometry.y.values
stations_1yr['canopy'] = canopy_ucrb.band_data.interp(x=xs, y=ys, method="nearest").values.diagonal()

In [20]:
canopy_bands = [[0,20], [20,40], [40,60], [60,80], [80,100]]

area_per_band = []
stations_per_band = []

for band in canopy_bands:
    sr_array = swe.SWE_Post[0].where((canopy_ucrb.band_data >= band[0]) & (canopy_ucrb.band_data < band[1]) & (swe.SWE_Post.mean(dim='Year')>0.1)).values.flatten()
    sr_array = sr_array[~np.isnan(sr_array)]
    area_per_band.append(len(sr_array)*(500**2)/1000000)
    stations_per_band.append(len(stations_1yr[(stations_1yr['canopy']>=band[0]) & (stations_1yr['canopy']<band[1])]))

data = {
        "canopy_band": canopy_bands,
        "area_per_band": area_per_band,
        "stations_per_band": stations_per_band
    }

#load data into a DataFrame object:
area_by_canopy = pd.DataFrame(data)

In [60]:
area_by_canopy['percent_stations'] = (area_by_canopy['stations_per_band']/area_by_canopy['stations_per_band'].sum())*100

In [22]:
canopy_area_by_elev0 = []
elev = elevation_bands[0]
for can in canopy_bands:
    sr_array = swe.SWE_Post[0].where((canopy_ucrb.band_data >= can[0]) & (canopy_ucrb.band_data < can[1]) & (swe.SWE_Post.mean(dim='Year')>0.1) & (snowmodel_dem >=elev[0]) & (snowmodel_dem <elev[1])).values.flatten()
    sr_array = sr_array[~np.isnan(sr_array)]
    canopy_area_by_elev0.append(len(sr_array)*(500**2)/1000000)
data = {
    "canopy_band": canopy_bands,
    "area_per_band": canopy_area_by_elev0,
    "elevation_band": [elev[1],elev[1],elev[1],elev[1],elev[1]]
}

#load data into a DataFrame object:
canopy_area_by_elev0_df = pd.DataFrame(data)

In [23]:
canopies = ['0-20','20-40','40-60','60-80','80-100']

canopy_area_by_elev0 = []
elev = elevation_bands[0]
for can in canopy_bands:
    sr_array = swe.SWE_Post[0].where((canopy_ucrb.band_data >= can[0]) & (canopy_ucrb.band_data < can[1]) & (swe.SWE_Post.mean(dim='Year')>0.1) & (snowmodel_dem >=elev[0]) & (snowmodel_dem <elev[1])).values.flatten()
    sr_array = sr_array[~np.isnan(sr_array)]
    canopy_area_by_elev0.append(len(sr_array)*(500**2)/1000000)
data = {
    "canopy_band": canopies,
    "area_per_band": canopy_area_by_elev0,
    "elevation_band": [elev[1],elev[1],elev[1],elev[1],elev[1]]
}

#load data into a DataFrame object:
canopy_area_by_elev0_df = pd.DataFrame(data)

canopy_area_by_elev1 = []
elev = elevation_bands[1]
for can in canopy_bands:
    sr_array = swe.SWE_Post[0].where((canopy_ucrb.band_data >= can[0]) & (canopy_ucrb.band_data < can[1]) & (swe.SWE_Post.mean(dim='Year')>0.1) & (snowmodel_dem >=elev[0]) & (snowmodel_dem <elev[1])).values.flatten()
    sr_array = sr_array[~np.isnan(sr_array)]
    canopy_area_by_elev1.append(len(sr_array)*(500**2)/1000000)
data = {
    "canopy_band": canopies,
    "area_per_band": canopy_area_by_elev1,
    "elevation_band": [elev[1],elev[1],elev[1],elev[1],elev[1]]
}

#load data into a DataFrame object:
canopy_area_by_elev1_df = pd.DataFrame(data)

canopy_area_by_elev2 = []
elev = elevation_bands[2]
for can in canopy_bands:
    sr_array = swe.SWE_Post[0].where((canopy_ucrb.band_data >= can[0]) & (canopy_ucrb.band_data < can[1]) & (swe.SWE_Post.mean(dim='Year')>0.1) & (snowmodel_dem >=elev[0]) & (snowmodel_dem <elev[1])).values.flatten()
    sr_array = sr_array[~np.isnan(sr_array)]
    canopy_area_by_elev2.append(len(sr_array)*(500**2)/1000000)
data = {
    "canopy_band": canopies,
    "area_per_band": canopy_area_by_elev2,
    "elevation_band": [elev[1],elev[1],elev[1],elev[1],elev[1]]
}

#load data into a DataFrame object:
canopy_area_by_elev2_df = pd.DataFrame(data)

canopy_area_by_elev3 = []
elev = elevation_bands[3]
for can in canopy_bands:
    sr_array = swe.SWE_Post[0].where((canopy_ucrb.band_data >= can[0]) & (canopy_ucrb.band_data < can[1]) & (swe.SWE_Post.mean(dim='Year')>0.1) & (snowmodel_dem >=elev[0]) & (snowmodel_dem <elev[1])).values.flatten()
    sr_array = sr_array[~np.isnan(sr_array)]
    canopy_area_by_elev3.append(len(sr_array)*(500**2)/1000000)
data = {
    "canopy_band": canopies,
    "area_per_band": canopy_area_by_elev3,
    "elevation_band": [elev[1],elev[1],elev[1],elev[1],elev[1]]
}

#load data into a DataFrame object:
canopy_area_by_elev3_df = pd.DataFrame(data)

canopy_area_by_elev4 = []
elev = elevation_bands[4]
for can in canopy_bands:
    sr_array = swe.SWE_Post[0].where((canopy_ucrb.band_data >= can[0]) & (canopy_ucrb.band_data < can[1]) & (swe.SWE_Post.mean(dim='Year')>0.1) & (snowmodel_dem >=elev[0]) & (snowmodel_dem <elev[1])).values.flatten()
    sr_array = sr_array[~np.isnan(sr_array)]
    canopy_area_by_elev4.append(len(sr_array)*(500**2)/1000000)
data = {
    "canopy_band": canopies,
    "area_per_band": canopy_area_by_elev4,
    "elevation_band": [elev[1],elev[1],elev[1],elev[1],elev[1]]
}

#load data into a DataFrame object:
canopy_area_by_elev4_df = pd.DataFrame(data)

# canopy_area_by_elev5 = []
# elev = elevation_bands[5]
# for can in canopy_bands:
#     sr_array = swe.SWE_Post[0].where((canopy_ucrb.band_data >= can[0]) & (canopy_ucrb.band_data < can[1]) & (swe.SWE_Post.mean(dim='Year')>0.1) & (snowmodel_dem >=elev[0]) & (snowmodel_dem <elev[1])).values.flatten()
#     sr_array = sr_array[~np.isnan(sr_array)]
#     canopy_area_by_elev5.append(len(sr_array)*(500**2)/1000000)
# data = {
#     "canopy_band": canopy_bands,
#     "area_per_band": canopy_area_by_elev5,
#     "elevation_band": [elev[1],elev[1],elev[1],elev[1],elev[1]]
# }

# #load data into a DataFrame object:
# canopy_area_by_elev5_df = pd.DataFrame(data)

metrics_canopy_elev = pd.concat(
    [
    canopy_area_by_elev0_df,
    canopy_area_by_elev1_df,
    canopy_area_by_elev2_df,
    canopy_area_by_elev3_df,
    canopy_area_by_elev4_df], 
    ignore_index=True)

In [139]:
metrics_canopy_elev

,canopy_band,area_per_band,elevation_band
0,"[0, 20]",7531.25,2500
1,"[20, 40]",1607.25,2500
2,"[40, 60]",1156.00,2500
3,"[60, 80]",915.00,2500
4,"[80, 100]",507.25,2500
5,"[0, 20]",7929.25,3000
6,"[20, 40]",5870.00,3000
7,"[40, 60]",6930.25,3000
8,"[60, 80]",8692.75,3000
9,"[80, 100]",7156.25,3000


In [150]:
# elev_chart = alt.Chart(area_by_elev).mark_bar().encode(
#     alt.Y("elevation_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None)),
#     alt.X('area_per_band', axis=alt.Axis(title ='Area in Each Band')),
#     color=alt.value('grey')
# ).properties(
#     width=250,
#     height=200
#     )

elev_can_chart = alt.Chart(metrics_canopy_elev).mark_bar().encode(
    alt.X("area_per_band:Q"),
    alt.Y("elevation_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(title='Elevation (m)')),
    alt.Color("canopy_band").scale(scheme="greens"),
    yOffset=alt.YOffset('canopy_band:O')
).properties(
    width=250,
    height=200
    )
elev_can_chart

alt.Chart(...)

In [132]:
metrics_canopy_elev

,canopy_band,area_per_band,elevation_band
0,"[0, 20]",7531.25,2500
1,"[20, 40]",1607.25,2500
2,"[40, 60]",1156.00,2500
3,"[60, 80]",915.00,2500
4,"[80, 100]",507.25,2500
5,"[0, 20]",7929.25,3000
6,"[20, 40]",5870.00,3000
7,"[40, 60]",6930.25,3000
8,"[60, 80]",8692.75,3000
9,"[80, 100]",7156.25,3000


In [63]:
#area_by_canopy['percent_area'] = (area_by_canopy['area_per_band']/(len(swe.SWE_Post[0].where(swe.SWE_Post.mean(dim='Year')>0.1).values.flatten())*(500**2)/1000000)) * 100

In [67]:
area_by_canopy['percent_area'] = (area_by_canopy['area_per_band']/77657)*100

In [65]:
77723.0 - 77657.0

66.0

In [68]:
area_by_canopy

,canopy_band,area_per_band,stations_per_band,percent_stations,percent_area
0,"[0, 20]",24702.25,33,50.000000,31.809431
1,"[20, 40]",10759.00,8,12.121212,13.854514
2,"[40, 60]",11907.75,2,3.030303,15.333775
3,"[60, 80]",14851.25,13,19.696970,19.124161
4,"[80, 100]",15436.75,10,15.151515,19.878118


In [73]:
area_by_canopy

,canopy_band,area_per_band,stations_per_band,percent_area
0,"[0, 20]",24702.25,33,31.809431
1,"[20, 40]",10759.00,8,13.854514
2,"[40, 60]",11907.75,2,15.333775
3,"[60, 80]",14851.25,13,19.124161
4,"[80, 100]",15436.75,10,19.878118


In [74]:
area_by_canopy['percent_area'].sum()

np.float64(100.00000000000001)

In [80]:
r_chart = alt.Chart(metrics_elev).mark_boxplot(extent=1).encode(
    alt.X("R:Q").scale(zero=False),
    alt.Y("elevation_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(title='Elevation (m)')),
    alt.Color("method"),
    yOffset=alt.YOffset('method:N')   #, scale=alt.Scale(domain=(10000,0))
).properties(
    width=250,
    height=200
    )

md_chart = alt.Chart(metrics_elev).mark_boxplot(extent=1).encode(
    alt.X("MD:Q").scale(zero=False),
    alt.Y("elevation_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None)),
    alt.Color("method"),
    yOffset=alt.YOffset('method:N')
).properties(
    width=250,
    height=200
    )

rmse_chart = alt.Chart(metrics_elev).mark_boxplot(extent=1).encode(
    alt.X("RMSE:Q").scale(zero=False),
    alt.Y("elevation_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None)),
    alt.Color("method", legend=alt.Legend(
        orient='none',
        legendX=375, legendY=-40,
        direction='horizontal',
        titleAnchor='middle')),
    yOffset=alt.YOffset('method:N')
).properties(
    width=250,
    height=200
    )

elev_chart = alt.Chart(area_by_elev).mark_bar().encode(
    alt.Y("elevation_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None)),
    alt.X('area_per_band', axis=alt.Axis(title ='Area in Each Band')),
    color=alt.value('grey')
).properties(
    width=250,
    height=200
    )

text_elev = elev_chart.mark_text(baseline="middle").encode(
    text = "stations_per_band:Q",
    color = alt.value('black')
)

# Canopy plot:
r_canopy = alt.Chart(metrics_canopy).mark_boxplot(extent=1).encode(
    alt.X("R:Q", scale=alt.Scale(domain=[0.1, 1.0])),
    alt.Y("canopy_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(title='Canopy (%)')),
    alt.Color("method"),
    yOffset=alt.YOffset('method:N')   #, scale=alt.Scale(domain=(10000,0))
).properties(
    width=250,
    height=200
    )

md_canopy = alt.Chart(metrics_canopy).mark_boxplot(extent=1).encode(
    alt.X("MD:Q", scale=alt.Scale(domain=[-0.25, 0.2])),
    alt.Y("canopy_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None)),
    alt.Color("method"),
    yOffset=alt.YOffset('method:N')
).properties(
    width=250,
    height=200
    )

rmse_canopy = alt.Chart(metrics_canopy).mark_boxplot(extent=1).encode(
    alt.X("RMSE:Q", scale=alt.Scale(domain=[0.04, 0.3])),
    alt.Y("canopy_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None)),
    alt.Color("method", legend=alt.Legend(
        orient='none',
        legendX=375, legendY=-40,
        direction='horizontal',
        titleAnchor='middle')),
    yOffset=alt.YOffset('method:N')
).properties(
    width=250,
    height=200
    )

canopy_chart = alt.Chart(area_by_canopy).mark_bar().encode(
    alt.Y("canopy_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None)),
    alt.X('area_per_band', axis=alt.Axis(title='Area in Each Band')),
    color=alt.value('grey')
).properties(
    width=250,
    height=200
    )

concat = alt.vconcat(alt.hconcat(r_chart, md_chart, rmse_chart, elev_chart + text_elev), alt.hconcat(r_canopy, md_canopy, rmse_canopy, canopy_chart))
concat

alt.VConcatChart(...)

In [28]:
r_chart = alt.Chart(metrics_elev).mark_boxplot(extent=1).encode(
    alt.X("R:Q", title='Correlation (R value)').scale(zero=False).axis(labelFontSize=14, titleFontSize=14),
    alt.Y("elevation_band:O", sort=['SA','Quantile'], scale=alt.Scale(reverse=True), axis=alt.Axis(title='Elevation (m)',labelFontSize=14, titleFontSize=14)),
    alt.Color("method", sort=['SA','Quantile']),
    yOffset=alt.YOffset('method:N')   #, scale=alt.Scale(domain=(10000,0))
).properties(
    width=250,
    height=200
    )

md_chart = alt.Chart(metrics_elev).mark_boxplot(extent=1).encode(
    alt.X("MD:Q").scale(zero=False).axis(labelFontSize=14, titleFontSize=14),
    alt.Y("elevation_band:O", sort=['SA','Quantile'], scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None,labelFontSize=14, titleFontSize=14)),
    alt.Color("method", sort=['SA','Quantile']),
    yOffset=alt.YOffset('method:N')
).properties(
    width=250,
    height=200
    )

rmse_chart = alt.Chart(metrics_elev).mark_boxplot(extent=1).encode(
    alt.X("RMSE:Q").scale(zero=False).axis(labelFontSize=14, titleFontSize=14),
    alt.Y("elevation_band:O", sort=['SA','Quantile'], scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None,labelFontSize=14, titleFontSize=14)),
    alt.Color("method", sort=['SA','Quantile'], legend=alt.Legend(
        orient='none',
        legendX=375, legendY=-40,
        direction='horizontal',
        titleAnchor='middle')),
    yOffset=alt.YOffset('method:N')
).properties(
    width=250,
    height=200
    )

elev_base = alt.Chart(area_by_elev).encode(
    alt.Y("elevation_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None,labelFontSize=14, titleFontSize=14))
)

elev_area = elev_base.mark_bar(color='#57A44C', opacity=0.5).encode(
    alt.X('area_per_band').axis(title='Area per Band', titleColor='#57A44C', labelColor='#57A44C',labelFontSize=14, titleFontSize=14) #
)

elev_stations = elev_base.mark_bar(color='#5276A7', interpolate='monotone', opacity=0.5).encode(
    alt.X('stations_per_band').axis(title='Stations per Band', titleColor='#5276A7', labelColor='#5276A7',labelFontSize=14, titleFontSize=14)
)

elev_chart = alt.layer(elev_area, elev_stations).resolve_scale(
    x='independent'
).properties(
    width=250,
    height=200
    )

# elev_chart = alt.Chart(area_by_elev).mark_bar().encode(
#     alt.Y("elevation_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None)),
#     alt.X('area_per_band', axis=alt.Axis(title ='Area in Each Band')),
#     color=alt.value('grey')
# ).properties(
#     width=250,
#     height=200
#     )

# text_elev = elev_chart.mark_text(baseline="middle").encode(
#     text = "stations_per_band:Q",
#     color = alt.value('black')
# )

# Canopy plot:
r_canopy = alt.Chart(metrics_canopy).mark_boxplot(extent=1).encode(
    alt.X("R:Q", scale=alt.Scale(domain=[0.1, 1.0])).axis(labelFontSize=14, titleFontSize=14),
    alt.Y("canopy_band:O", sort=['SA','Quantile'], scale=alt.Scale(reverse=True), axis=alt.Axis(title='Canopy (%)',labelFontSize=14, titleFontSize=14)),
    alt.Color("method", sort=['SA','Quantile']),
    yOffset=alt.YOffset('method:N')   #, scale=alt.Scale(domain=(10000,0))
).properties(
    width=250,
    height=200
    )

md_canopy = alt.Chart(metrics_canopy).mark_boxplot(extent=1).encode(
    alt.X("MD:Q", scale=alt.Scale(domain=[-0.25, 0.2])).axis(labelFontSize=14, titleFontSize=14),
    alt.Y("canopy_band:O", sort=['SA','Quantile'], scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None,labelFontSize=14, titleFontSize=14)),
    alt.Color("method", sort=['SA','Quantile']),
    yOffset=alt.YOffset('method:N')
).properties(
    width=250,
    height=200
    )

rmse_canopy = alt.Chart(metrics_canopy).mark_boxplot(extent=1).encode(
    alt.X("RMSE:Q", scale=alt.Scale(domain=[0.04, 0.3])).axis(labelFontSize=14, titleFontSize=14),
    alt.Y("canopy_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None,labelFontSize=14, titleFontSize=14), sort=['SA','Quantile']),
    alt.Color("method", sort=['SA','Quantile'], legend=alt.Legend(
        orient='none',
        legendX=375, legendY=-40,
        direction='horizontal',
        titleAnchor='middle')),
    yOffset=alt.YOffset('method:N')
).properties(
    width=250,
    height=200
)

canopy_base = alt.Chart(area_by_canopy).encode(
    alt.Y("canopy_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None,labelFontSize=14, titleFontSize=14))
)

canopy_area = canopy_base.mark_bar(color='#57A44C', opacity=0.5).encode(
    alt.X('area_per_band').axis(title='Area per Band', titleColor='#57A44C', labelColor='#57A44C',labelFontSize=14, titleFontSize=14) #
)

canopy_stations = canopy_base.mark_bar(color='#5276A7', interpolate='monotone', opacity=0.5).encode(
    alt.X('stations_per_band').axis(title='Stations per Band', titleColor='#5276A7', labelColor='#5276A7',labelFontSize=14, titleFontSize=14)
)

canopy_chart = alt.layer(canopy_area, canopy_stations).resolve_scale(
    x='independent'
).properties(
    width=250,
    height=200
    )

# canopy_chart = alt.Chart(area_by_canopy).mark_bar().encode(
#     alt.Y("canopy_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None)),
#     alt.X('area_per_band', axis=alt.Axis(title='Area in Each Band')),
#     color=alt.value('grey')
# ).properties(
#     width=250,
#     height=200
#     )

concat = alt.vconcat(alt.hconcat(r_chart, md_chart, rmse_chart, elev_chart), alt.hconcat(r_canopy, md_canopy, rmse_canopy, canopy_chart))
concat

alt.VConcatChart(...)

In [31]:
r_chart = alt.Chart(metrics_elev).mark_boxplot(extent=1).encode(
    alt.X("R:Q", title='Correlation (R value)').scale(zero=False).axis(labelFontSize=14, titleFontSize=14),
    alt.Y("elevation_band:O", sort=['SA','Quantile'], scale=alt.Scale(reverse=True), axis=alt.Axis(title='Elevation (m)',labelFontSize=14, titleFontSize=14)),
    alt.Color("method", sort=['SA','Quantile']),
    yOffset=alt.YOffset('method:N')   #, scale=alt.Scale(domain=(10000,0))
).properties(
    width=250,
    height=200
    )

md_chart = alt.Chart(metrics_elev).mark_boxplot(extent=1).encode(
    alt.X("MD:Q").scale(zero=False).axis(labelFontSize=14, titleFontSize=14),
    alt.Y("elevation_band:O", sort=['SA','Quantile'], scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None,labelFontSize=14, titleFontSize=14)),
    alt.Color("method", sort=['SA','Quantile']),
    yOffset=alt.YOffset('method:N')
).properties(
    width=250,
    height=200
    )

rmse_chart = alt.Chart(metrics_elev).mark_boxplot(extent=1).encode(
    alt.X("RMSE:Q").scale(zero=False).axis(labelFontSize=14, titleFontSize=14),
    alt.Y("elevation_band:O", sort=['SA','Quantile'], scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None,labelFontSize=14, titleFontSize=14)),
    alt.Color("method", sort=['SA','Quantile'], legend=alt.Legend(
        orient='none',
        legendX=375, legendY=-40,
        direction='horizontal',
        titleAnchor='middle')),
    yOffset=alt.YOffset('method:N')
).properties(
    width=250,
    height=200
    )

elev_base = alt.Chart(area_by_elev).encode(
    alt.Y("elevation_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None,labelFontSize=14, titleFontSize=14))
)

elev_area = elev_base.mark_line(point=True, color='#57A44C').encode(
    alt.X('percent_area').axis(title='Percent Area per Band', titleColor='#57A44C', labelColor='#57A44C',labelFontSize=14, titleFontSize=14) #
)

elev_stations = elev_base.mark_line(point=True, color='#5276A7').encode(
    alt.X('stations_per_band').axis(title='Stations per Band', titleColor='#5276A7', labelColor='#5276A7',labelFontSize=14, titleFontSize=14)
)

elev_chart = alt.layer(elev_area, elev_stations).resolve_scale(
    x='independent'
).properties(
    width=250,
    height=200
    )

# elev_chart = alt.Chart(area_by_elev).mark_bar().encode(
#     alt.Y("elevation_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None)),
#     alt.X('area_per_band', axis=alt.Axis(title ='Area in Each Band')),
#     color=alt.value('grey')
# ).properties(
#     width=250,
#     height=200
#     )

# text_elev = elev_chart.mark_text(baseline="middle").encode(
#     text = "stations_per_band:Q",
#     color = alt.value('black')
# )

# Canopy plot:
r_canopy = alt.Chart(metrics_canopy).mark_boxplot(extent=1).encode(
    alt.X("R:Q", scale=alt.Scale(domain=[0.1, 1.0])).axis(labelFontSize=14, titleFontSize=14),
    alt.Y("canopy_band:O", sort=['SA','Quantile'], scale=alt.Scale(reverse=True), axis=alt.Axis(title='Canopy (%)',labelFontSize=14, titleFontSize=14)),
    alt.Color("method", sort=['SA','Quantile']),
    yOffset=alt.YOffset('method:N')   #, scale=alt.Scale(domain=(10000,0))
).properties(
    width=250,
    height=200
    )

md_canopy = alt.Chart(metrics_canopy).mark_boxplot(extent=1).encode(
    alt.X("MD:Q", scale=alt.Scale(domain=[-0.25, 0.2])).axis(labelFontSize=14, titleFontSize=14),
    alt.Y("canopy_band:O", sort=['SA','Quantile'], scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None,labelFontSize=14, titleFontSize=14)),
    alt.Color("method", sort=['SA','Quantile']),
    yOffset=alt.YOffset('method:N')
).properties(
    width=250,
    height=200
    )

rmse_canopy = alt.Chart(metrics_canopy).mark_boxplot(extent=1).encode(
    alt.X("RMSE:Q", scale=alt.Scale(domain=[0.04, 0.3])).axis(labelFontSize=14, titleFontSize=14),
    alt.Y("canopy_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None,labelFontSize=14, titleFontSize=14), sort=['SA','Quantile']),
    alt.Color("method", sort=['SA','Quantile'], legend=alt.Legend(
        orient='none',
        legendX=375, legendY=-40,
        direction='horizontal',
        titleAnchor='middle')),
    yOffset=alt.YOffset('method:N')
).properties(
    width=250,
    height=200
)

canopy_base = alt.Chart(area_by_canopy).encode(
    alt.Y("canopy_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None,labelFontSize=14, titleFontSize=14))
)

canopy_area = canopy_base.mark_line(point=False, color='#57A44C').encode(
    alt.X('percent_area').axis(title='Percent Area per Band', titleColor='#57A44C', labelColor='#57A44C',labelFontSize=14, titleFontSize=14), 
    shape=alt.Shape() #
)

canopy_area_pts = canopy_base.mark_point(shape='triangle', color='#57A44C').encode(
    alt.X('percent_area').axis(title='Percent Area per Band', titleColor='#57A44C', labelColor='#57A44C',labelFontSize=14, titleFontSize=14), 
    shape=alt.Shape() #
)

canopy_stations = canopy_base.mark_line(point=True, color='#5276A7').encode(
    alt.X('stations_per_band').axis(title='Stations per Band', titleColor='#5276A7', labelColor='#5276A7',labelFontSize=14, titleFontSize=14)
)

canopy_chart = alt.layer(canopy_area, canopy_stations).resolve_scale(
    x='independent'
).properties(
    width=250,
    height=200
    )

# canopy_chart = alt.Chart(area_by_canopy).mark_bar().encode(
#     alt.Y("canopy_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None)),
#     alt.X('area_per_band', axis=alt.Axis(title='Area in Each Band')),
#     color=alt.value('grey')
# ).properties(
#     width=250,
#     height=200
#     )

concat = alt.vconcat(alt.hconcat(r_chart, rmse_chart, elev_chart), alt.hconcat(r_canopy, rmse_canopy, canopy_chart))
concat

alt.VConcatChart(...)

In [89]:
canopy_base = alt.Chart(area_by_canopy).encode(
    alt.Y("canopy_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None,labelFontSize=14, titleFontSize=14))
)

canopy_area = canopy_base.mark_line(point=False, color='#a55194').encode(
    alt.X('percent_area').axis(title='Percent Area per Band', titleColor='#a55194', labelColor='#a55194',labelFontSize=14, titleFontSize=14).scale(domain=(0,60)), 
    shape=alt.Shape() #
)

canopy_area_pts = canopy_base.mark_point(shape='square', color='#a55194', size=130, filled=True, fillOpacity=1).encode(
    alt.X('percent_area').scale(domain=(0,60)).axis(labels=False, title='', tickSize=0))

canopy_stations = canopy_base.mark_line(point=False, color='#31a354').encode(
    alt.X('percent_stations').scale(domain=(0,60)).axis(title='Percent Stations per Band', titleColor='#31a354', labelColor='#31a354',labelFontSize=14, titleFontSize=14)
)

canopy_stations_pts = canopy_base.mark_point(shape='circle', color='#31a354', size=130, filled=True, fillOpacity=1).encode(
    alt.X('percent_stations').scale(domain=(0,60)).axis(labels=False, title='', tickSize=0))

canopy_chart = alt.layer(canopy_area, canopy_area_pts, canopy_stations, canopy_stations_pts).resolve_scale(
    x='independent'
).properties(
    width=250,
    height=200
    )

canopy_chart

alt.LayerChart(...)

In [99]:
r_chart = alt.Chart(metrics_elev).mark_boxplot(extent=1).encode(
    alt.X("R:Q", title='Correlation (R value)').scale(zero=False).axis(labelFontSize=14, titleFontSize=14),
    alt.Y("elevation_band:O", sort=['SA','Quantile'], scale=alt.Scale(reverse=True), axis=alt.Axis(title='Elevation (m)',labelFontSize=14, titleFontSize=14)),
    alt.Color("method", sort=['SA','Quantile']),
    yOffset=alt.YOffset('method:N')   #, scale=alt.Scale(domain=(10000,0))
).properties(
    width=250,
    height=200
    )

md_chart = alt.Chart(metrics_elev).mark_boxplot(extent=1).encode(
    alt.X("MD:Q").scale(zero=False).axis(labelFontSize=14, titleFontSize=14),
    alt.Y("elevation_band:O", sort=['SA','Quantile'], scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None,labelFontSize=14, titleFontSize=14)),
    alt.Color("method", sort=['SA','Quantile']),
    yOffset=alt.YOffset('method:N')
).properties(
    width=250,
    height=200
    )

rmse_chart = alt.Chart(metrics_elev).mark_boxplot(extent=1).encode(
    alt.X("RMSE:Q").scale(zero=False).axis(labelFontSize=14, titleFontSize=14),
    alt.Y("elevation_band:O", sort=['SA','Quantile'], scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None,labelFontSize=14, titleFontSize=14)),
    alt.Color("method", sort=['SA','Quantile'], legend=alt.Legend(False)),
    yOffset=alt.YOffset('method:N')
).properties(
    width=250,
    height=200
    )

# , legend=alt.Legend(
#         orient='none',
#         legendX=375, legendY=-40,
#         direction='horizontal',
#         titleAnchor='middle')

# elev_base = alt.Chart(area_by_elev).encode(
#     alt.Y("elevation_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None,labelFontSize=14, titleFontSize=14))
# )

# elev_area = elev_base.mark_line(color='#a55194').encode(
#     alt.X('percent_area', scale=alt.Scale(domain=[0, 60])).axis(title='Percent Area per Elevation Band', titleColor='#a55194', labelColor='#a55194',labelFontSize=14, titleFontSize=14) #
# )

# elev_area = elev_area + elev_area.mark_circle(color='#a55194')

# elev_stations = elev_base.mark_line(point=True, color='#31a354').encode(
#     alt.X('percent_stations', scale=alt.Scale(domain=[0, 60])).axis(title='Percent Stations per Elevation Band', titleColor='#31a354', labelColor='#31a354',labelFontSize=14, titleFontSize=14)
# )

# elev_stations = elev_stations + elev_stations.mark_square(color='#31a354')

# elev_chart = alt.layer(elev_area, elev_stations).resolve_scale(
#     x='independent'
# ).properties(
#     width=250,
#     height=200
#     )

elev_base = alt.Chart(area_by_elev).encode(
    alt.Y("elevation_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None,labelFontSize=14, titleFontSize=14))
)

elev_area = elev_base.mark_line(point=False, color='#a55194').encode(
    alt.X('percent_area').axis(title='Percent Area per Elevation Band', titleColor='#a55194', labelColor='#a55194',labelFontSize=14, titleFontSize=14).scale(domain=(0,60)), 
    shape=alt.Shape() #
)

elev_area_pts = elev_base.mark_point(shape='square', color='#a55194', size=130, filled=True, fillOpacity=1).encode(
    alt.X('percent_area').scale(domain=(0,60)).axis(labels=False, title='', tickSize=0))

elev_stations = elev_base.mark_line(point=False, color='#31a354').encode(
    alt.X('percent_stations').scale(domain=(0,60)).axis(title='Percent Stations per Elevation Band', titleColor='#31a354', labelColor='#31a354',labelFontSize=14, titleFontSize=14)
)

elev_stations_pts = elev_base.mark_point(shape='circle', color='#31a354', size=130, filled=True, fillOpacity=1).encode(
    alt.X('percent_stations').scale(domain=(0,60)).axis(labels=False, title='', tickSize=0))

elev_chart = alt.layer(elev_area, elev_area_pts, elev_stations, elev_stations_pts).resolve_scale(
    x='independent'
).properties(
    width=250,
    height=200
    )
# )

# Canopy plot:
r_canopy = alt.Chart(metrics_canopy).mark_boxplot(extent=1).encode(
    alt.X("R:Q", scale=alt.Scale(domain=[0.1, 1.0])).axis(labelFontSize=14, titleFontSize=14),
    alt.Y("canopy_band:O", sort=['SA','Quantile'], scale=alt.Scale(reverse=True), axis=alt.Axis(title='Canopy (%)',labelFontSize=14, titleFontSize=14)),
    alt.Color("method", sort=['SA','Quantile']),
    yOffset=alt.YOffset('method:N')   #, scale=alt.Scale(domain=(10000,0))
).properties(
    width=250,
    height=200
    )

md_canopy = alt.Chart(metrics_canopy).mark_boxplot(extent=1).encode(
    alt.X("MD:Q", scale=alt.Scale(domain=[-0.25, 0.2])).axis(labelFontSize=14, titleFontSize=14),
    alt.Y("canopy_band:O", sort=['SA','Quantile'], scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None,labelFontSize=14, titleFontSize=14)),
    alt.Color("method", sort=['SA','Quantile']),
    yOffset=alt.YOffset('method:N')
).properties(
    width=250,
    height=200
    )

rmse_canopy = alt.Chart(metrics_canopy).mark_boxplot(extent=1).encode(
    alt.X("RMSE:Q", scale=alt.Scale(domain=[0.04, 0.3])).axis(labelFontSize=14, titleFontSize=14),
    alt.Y("canopy_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None,labelFontSize=14, titleFontSize=14), sort=['SA','Quantile']),
    alt.Color("method", sort=['SA','Quantile'], legend=alt.Legend(
        orient='none',
        legendX=375, legendY=-40,
        direction='horizontal',
        titleAnchor='middle')),
    yOffset=alt.YOffset('method:N')
).properties(
    width=250,
    height=200
)

canopy_base = alt.Chart(area_by_canopy).encode(
    alt.Y("canopy_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None,labelFontSize=14, titleFontSize=14))
)

canopy_area = canopy_base.mark_line(point=False, color='#a55194').encode(
    alt.X('percent_area').axis(title='Percent Area per Band', titleColor='#a55194', labelColor='#a55194',labelFontSize=14, titleFontSize=14).scale(domain=(0,60)), 
    shape=alt.Shape() #
)

canopy_area_pts = canopy_base.mark_point(shape='square', color='#a55194', size=130, filled=True, fillOpacity=1).encode(
    alt.X('percent_area').scale(domain=(0,60)).axis(labels=False, title='', tickSize=0))

canopy_stations = canopy_base.mark_line(point=False, color='#31a354').encode(
    alt.X('percent_stations').scale(domain=(0,60)).axis(title='Percent Stations per Band', titleColor='#31a354', labelColor='#31a354',labelFontSize=14, titleFontSize=14)
)

canopy_stations_pts = canopy_base.mark_point(shape='circle', color='#31a354', size=130, filled=True, fillOpacity=1).encode(
    alt.X('percent_stations').scale(domain=(0,60)).axis(labels=False, title='', tickSize=0))

canopy_chart = alt.layer(canopy_area, canopy_area_pts, canopy_stations, canopy_stations_pts).resolve_scale(
    x='independent'
).properties(
    width=250,
    height=200
    )

concat = alt.vconcat(alt.hconcat(r_chart, rmse_chart, elev_chart), alt.hconcat(r_canopy, rmse_canopy, canopy_chart))
concat

alt.VConcatChart(...)

In [96]:
concat.save('../../figures/metrics_elev_canopy_3242025.png', ppi=500)

In [75]:
canopy_base = alt.Chart(area_by_canopy).encode(
    alt.Y("canopy_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None,labelFontSize=14, titleFontSize=14))
)

canopy_area = canopy_base.mark_line(point=True, color='#57A44C').encode(
    alt.X('percent_area').axis(title='Percent Area per Band', titleColor='#57A44C', labelColor='#57A44C',labelFontSize=14, titleFontSize=14) #
)

canopy_stations = canopy_base.mark_line(point=True, color='#5276A7').encode(
    alt.X('stations_per_band').axis(title='Stations per Band', titleColor='#5276A7', labelColor='#5276A7',labelFontSize=14, titleFontSize=14)
)

canopy_chart = alt.layer(canopy_area, canopy_stations).resolve_scale(
    x='independent'
).properties(
    width=250,
    height=200
    )

canopy_chart

alt.LayerChart(...)

In [29]:
concat.save('../../figures/metrics_elev_canopy_342025.png', ppi=500)

In [110]:
elev_base = alt.Chart(area_by_elev).encode(
    alt.Y("elevation_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None))
)

elev_area = elev_base.mark_bar(color='#57A44C', opacity=0.5).encode(
    alt.X('area_per_band').axis(title='Area in Each Band', titleColor='#57A44C',labelFontSize=14, titleFontSize=14) #
)

elev_stations = elev_base.mark_bar(color='#5276A7', interpolate='monotone', opacity=0.5).encode(
    alt.X('stations_per_band').axis(title='Stations per Band', titleColor='#5276A7',labelFontSize=14, titleFontSize=14)
)

elev_chart = alt.layer(elev_area, elev_stations).resolve_scale(
    x='independent'
)

elev_chart

alt.LayerChart(...)

In [ ]:
elev_base = alt.Chart(area_by_elev).encode(
    alt.Y("elevation_band:O", scale=alt.Scale(reverse=True), axis=alt.Axis(labels=False, title=None))
)

elev_area = elev_base.mark_bar(color='#57A44C', opacity=0.5).encode(
    alt.X('area_per_band').axis(title='Area in Each Band', titleColor='#57A44C') #
)

elev_stations = elev_base.mark_bar(color='#5276A7', interpolate='monotone', opacity=0.5).encode(
    alt.X('stations_per_band').axis(title='Stations per Band', titleColor='#5276A7')
)

elev_chart = alt.layer(elev_area, elev_stations).resolve_scale(
    x='independent'
)

elev_chart